# S&P500 적용

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pandas_datareader import data as pdr
import fix_yahoo_finance
sp500 = fix_yahoo_finance.download("^GSPC", start="2011-01-01", end="2018-10-30")

[*********************100%***********************]  1 of 1 downloaded


In [53]:
ks11 = fix_yahoo_finance.download("^KS11", start="2009-01-01", end="2018-10-30")

[*********************100%***********************]  1 of 1 downloaded


In [54]:
ks11.to_csv('ks11.csv')

In [5]:
sp500.to_csv('s&p500.csv')
ks11.to_csv('ks11.csv')

In [6]:
pd.concat((ks11.Close, ks11.Close.shift(5).rolling(3).mean(), ks11.Close - ks11.Close.shift(5).rolling(3).mean(),
           (ks11.Close - ks11.Close.shift(5).rolling(3).mean()).shift(-5)), axis=1).head(10)

,Close,Close,Close,Close
Date,,,,
2011-01-03,2070.080078,NaN,NaN,NaN
2011-01-04,2085.139893,NaN,NaN,NaN
2011-01-05,2082.550049,NaN,NaN,15.693278
2011-01-06,2077.610107,NaN,NaN,7.713297
2011-01-07,2086.199951,NaN,NaN,26.049886
2011-01-10,2080.810059,NaN,NaN,18.310059
2011-01-11,2088.320068,NaN,NaN,11.369954
2011-01-12,2094.949951,2079.256673,15.693278,27.663248
2011-01-13,2089.479980,2081.766683,7.713297,15.743246


In [7]:
(ks11.Close - ks11.Close.shift(5).rolling(3).mean()).shift(-5).head()

Date
2011-01-03          NaN
2011-01-04          NaN
2011-01-05    15.693278
2011-01-06     7.713297
2011-01-07    26.049886
Name: Close, dtype: float64

In [20]:
sp500 = fix_yahoo_finance.download("^GSPC", start="2008-01-01", end="2018-10-30")

sp500["pct_change"] = sp500.Close.pct_change()
for i in range(3, 31, 2):
    sp500["close_ma_{}".format(i)] = sp500.Close.rolling(window = i).mean()
for i in [7, 21, 30, 60, 90, 91, 100, 120]:
    sp500["his_vol_{}".format(i)] = sp500["pct_change"].rolling(window = i).std()*(252**0.5)
for i in [1, 3, 5, 7, 10, 12, 14]:
    sp500['returnsCC{}'.format(i) ]= np.log(sp500.Open / sp500.Close.shift(i))
    sp500['returnsOO{}'.format(i) ] = np.log(sp500.Open / sp500.Open.shift(i))
    sp500['returnsCC{}'.format(i) ]= np.log(sp500.Close / sp500.Close.shift(i))
    sp500['returnsOC{}'.format(i) ] = np.log(sp500.Close / sp500.Open.shift(i))
for i in [1, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50,
          55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]:
    for shift in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        sp500['Y1_{}_s{}'.format(i, shift)] = (sp500.Close - sp500.Close.shift(shift).rolling(i).mean())
        sp500['Y3_{}_s{}'.format(i, shift)] = (sp500.Close.rolling(i).mean() - sp500.Close.shift(shift).rolling(i).mean())
        sp500['sh{}_Y1_{}'.format(shift, i)] = np.where((sp500['Y1_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        sp500['sh{}_Y3_{}'.format(shift, i)] = np.where((sp500['Y3_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)

[*********************100%***********************]  1 of 1 downloaded


In [60]:
ks11 = pd.read_csv('ks11.csv')

In [63]:
ks11.index = ks11.Date
del ks11['Date']
ks11

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,1132.869995,1157.400024,1118.839966,1157.400024,1157.400024,340600
2009-01-05,1180.979980,1185.219971,1166.660034,1173.569946,1173.569946,414600
2009-01-06,1183.520020,1203.589966,1179.939941,1194.280029,1194.280029,444000
2009-01-07,1198.849976,1228.560059,1195.199951,1228.170044,1228.170044,568400
2009-01-08,1212.699951,1218.430054,1195.150024,1205.699951,1205.699951,472200
2009-01-09,1214.500000,1220.589966,1179.060059,1180.959961,1180.959961,443200
2009-01-12,1170.689941,1177.189941,1150.410034,1156.750000,1156.750000,352600
2009-01-13,1147.989990,1168.040039,1133.939941,1167.709961,1167.709961,381200
2009-01-14,1161.010010,1184.520020,1151.800049,1182.680054,1182.680054,358600


In [70]:
data = ks11.copy()
data["pct_change"] = data.Close.pct_change()
for i in range(3, 31, 2):
    data["close_ma_{}".format(i)] = data.Close.rolling(window = i).mean()
for i in [7, 21, 30, 60, 90, 91, 100, 120]:
    data["his_vol_{}".format(i)] = data["pct_change"].rolling(window = i).std()*(252**0.5)
for i in [1, 3, 5, 7, 10, 12, 14]:
    data['returnsCC{}'.format(i) ]= np.log(data.Open / data.Close.shift(i))
    data['returnsOO{}'.format(i) ] = np.log(data.Open / data.Open.shift(i))
    data['returnsCC{}'.format(i) ]= np.log(data.Close / data.Close.shift(i))
    data['returnsOC{}'.format(i) ] = np.log(data.Close / data.Open.shift(i))
for i in [1, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50,
          55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]:
    for shift in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
        data['Y1_{}_s{}'.format(i, shift)] = (data.Close - data.Close.shift(shift).rolling(i).mean())
        data['Y3_{}_s{}'.format(i, shift)] = (data.Close.rolling(i).mean() - data.Close.shift(shift).rolling(i).mean())
        data['sh{}_Y1_{}'.format(shift, i)] = np.where((data['Y1_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        data['sh{}_Y3_{}'.format(shift, i)] = np.where((data['Y3_{}_s{}'.format(i, shift)]).shift(-1*shift)>=0, 1, 0)
        
data_ = data.loc['2018-07-01':'2018-10-01']
data = data.loc['2013-07-01':'2018-07-01']

In [72]:
ks11

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,1132.869995,1157.400024,1118.839966,1157.400024,1157.400024,340600
2009-01-05,1180.979980,1185.219971,1166.660034,1173.569946,1173.569946,414600
2009-01-06,1183.520020,1203.589966,1179.939941,1194.280029,1194.280029,444000
2009-01-07,1198.849976,1228.560059,1195.199951,1228.170044,1228.170044,568400
2009-01-08,1212.699951,1218.430054,1195.150024,1205.699951,1205.699951,472200
2009-01-09,1214.500000,1220.589966,1179.060059,1180.959961,1180.959961,443200
2009-01-12,1170.689941,1177.189941,1150.410034,1156.750000,1156.750000,352600
2009-01-13,1147.989990,1168.040039,1133.939941,1167.709961,1167.709961,381200
2009-01-14,1161.010010,1184.520020,1151.800049,1182.680054,1182.680054,358600


In [21]:
sp500_test = sp500.loc['2018-07-01':'2018-10-01']
sp500 = sp500.loc['2013-07-01':'2018-07-01']

In [23]:
X_li = [i for i in sp500.columns if (i.find('sh') == -1) if (i.find('Volume') == -1)]

In [24]:
%run moon_code

In [25]:
train, test = train_test_split(sp500, 0.8, dtrain=True)

In [26]:
train

,Open,High,Low,Close,Adj Close,Volume,pct_change,close_ma_3,close_ma_5,close_ma_7,...,sh8_Y1_120,sh8_Y3_120,Y1_120_s9,Y3_120_s9,sh9_Y1_120,sh9_Y3_120,Y1_120_s10,Y3_120_s10,sh10_Y1_120,sh10_Y3_120
Date,,,,,,,,,,,,,,,,,,,,,
2013-07-01,1609.780029,1626.609985,1609.780029,1614.959961,1614.959961,-1190277296,0.005404,1611.479980,1605.145996,1598.750000,...,1,1,58.965130,11.954333,1,1,60.841380,13.830584,1,1
2013-07-02,1614.290039,1624.260010,1606.770020,1614.079956,1614.079956,-977837296,-0.000545,1611.773315,1610.355981,1601.842843,...,1,1,56.342624,11.487332,1,1,58.085125,13.229833,1,1
2013-07-03,1611.479980,1618.969971,1604.569946,1615.410034,1615.410034,1966050000,0.000824,1614.816650,1612.785986,1607.888567,...,1,1,56.255285,11.263999,1,1,57.672702,12.681416,1,1
2013-07-05,1618.650024,1632.069946,1614.709961,1631.890015,1631.890015,-1660827296,0.010202,1620.460002,1616.523999,1614.154279,...,1,1,71.151933,11.012666,1,1,72.735266,12.595999,1,1
2013-07-08,1634.199951,1644.680054,1634.199951,1640.459961,1640.459961,-780377296,0.005252,1629.253337,1623.359985,1619.468558,...,1,1,78.497712,11.203331,1,1,79.721879,12.427498,1,1
2013-07-09,1642.890015,1654.180054,1642.890015,1652.319946,1652.319946,-1139607296,0.007230,1641.556641,1630.831982,1625.057129,...,1,1,89.310947,11.656415,1,1,90.357697,12.703165,1,1
2013-07-10,1651.560059,1657.920044,1647.660034,1652.619995,1652.619995,-1283957296,0.000182,1648.466634,1638.539990,1631.677124,...,1,1,88.411913,11.957248,1,1,89.610996,13.156331,1,1
2013-07-11,1657.410034,1676.630005,1657.410034,1675.020020,1675.020020,-848627296,0.013554,1659.986654,1650.461987,1640.257132,...,1,1,109.589188,12.351832,1,1,110.811938,13.574582,1,1
2013-07-12,1675.260010,1680.189941,1672.329956,1680.189941,1680.189941,-1255897296,0.003086,1669.276652,1660.121973,1649.701416,...,1,1,113.555859,12.766998,1,1,114.759109,13.970248,1,1


# Learning

In [27]:
y_li = [i for i in sp500.columns if i.find('sh') != -1]

In [28]:
%run xgb_modelfit

In [29]:
import copy

def modelfit(alg, train, test, X, y):
    alg = copy.deepcopy(alg)
    xgb_params = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(train[X].values, label=train[y].values)
    cvresult=xgb.cv(xgb_params, xgtrain,
                    num_boost_round=alg.get_params()['n_estimators'],
                    nfold=5, early_stopping_rounds=50,
                    show_stdv=False)
    alg.set_params(n_estimators=cvresult.shape[0])
    alg.fit(train[X], train[y], eval_metric='auc')
    dtrain_predictions = alg.predict(train[X])
    dtrain_predprob = alg.predict_proba(train[X])[:,1]
    dtest_predictions = alg.predict(test[X])
    dtest_predprob = alg.predict_proba(test[X])[:,1]
    print("Train Accuracy : {:.4f}".format(metrics.accuracy_score(
                            train[y].values, dtrain_predictions)))
    print("Train AUC Score : {:.4f}".format(metrics.roc_auc_score(
                            train[y].values, dtrain_predprob)))
    print("Test  Accuracy : {:.4f}".format(metrics.accuracy_score(
                            test[y].values, dtest_predictions)))
    print("Test  AUC Score : {:.4f}".format(metrics.roc_auc_score(
                            test[y].values, dtest_predprob)))
    return alg

In [30]:
X_li

['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'pct_change',
 'close_ma_3',
 'close_ma_5',
 'close_ma_7',
 'close_ma_9',
 'close_ma_11',
 'close_ma_13',
 'close_ma_15',
 'close_ma_17',
 'close_ma_19',
 'close_ma_21',
 'close_ma_23',
 'close_ma_25',
 'close_ma_27',
 'close_ma_29',
 'his_vol_7',
 'his_vol_21',
 'his_vol_30',
 'his_vol_60',
 'his_vol_90',
 'his_vol_91',
 'his_vol_100',
 'his_vol_120',
 'returnsCC1',
 'returnsOO1',
 'returnsOC1',
 'returnsCC3',
 'returnsOO3',
 'returnsOC3',
 'returnsCC5',
 'returnsOO5',
 'returnsOC5',
 'returnsCC7',
 'returnsOO7',
 'returnsOC7',
 'returnsCC10',
 'returnsOO10',
 'returnsOC10',
 'returnsCC12',
 'returnsOO12',
 'returnsOC12',
 'returnsCC14',
 'returnsOO14',
 'returnsOC14',
 'Y1_1_s1',
 'Y3_1_s1',
 'Y1_1_s2',
 'Y3_1_s2',
 'Y1_1_s3',
 'Y3_1_s3',
 'Y1_1_s4',
 'Y3_1_s4',
 'Y1_1_s5',
 'Y3_1_s5',
 'Y1_1_s6',
 'Y3_1_s6',
 'Y1_1_s7',
 'Y3_1_s7',
 'Y1_1_s8',
 'Y3_1_s8',
 'Y1_1_s9',
 'Y3_1_s9',
 'Y1_1_s10',
 'Y3_1_s10',
 'Y1_3_s1',
 'Y3_3_s1',
 'Y1

In [31]:
y_li

['sh1_Y1_1',
 'sh1_Y3_1',
 'sh2_Y1_1',
 'sh2_Y3_1',
 'sh3_Y1_1',
 'sh3_Y3_1',
 'sh4_Y1_1',
 'sh4_Y3_1',
 'sh5_Y1_1',
 'sh5_Y3_1',
 'sh6_Y1_1',
 'sh6_Y3_1',
 'sh7_Y1_1',
 'sh7_Y3_1',
 'sh8_Y1_1',
 'sh8_Y3_1',
 'sh9_Y1_1',
 'sh9_Y3_1',
 'sh10_Y1_1',
 'sh10_Y3_1',
 'sh1_Y1_3',
 'sh1_Y3_3',
 'sh2_Y1_3',
 'sh2_Y3_3',
 'sh3_Y1_3',
 'sh3_Y3_3',
 'sh4_Y1_3',
 'sh4_Y3_3',
 'sh5_Y1_3',
 'sh5_Y3_3',
 'sh6_Y1_3',
 'sh6_Y3_3',
 'sh7_Y1_3',
 'sh7_Y3_3',
 'sh8_Y1_3',
 'sh8_Y3_3',
 'sh9_Y1_3',
 'sh9_Y3_3',
 'sh10_Y1_3',
 'sh10_Y3_3',
 'sh1_Y1_5',
 'sh1_Y3_5',
 'sh2_Y1_5',
 'sh2_Y3_5',
 'sh3_Y1_5',
 'sh3_Y3_5',
 'sh4_Y1_5',
 'sh4_Y3_5',
 'sh5_Y1_5',
 'sh5_Y3_5',
 'sh6_Y1_5',
 'sh6_Y3_5',
 'sh7_Y1_5',
 'sh7_Y3_5',
 'sh8_Y1_5',
 'sh8_Y3_5',
 'sh9_Y1_5',
 'sh9_Y3_5',
 'sh10_Y1_5',
 'sh10_Y3_5',
 'sh1_Y1_7',
 'sh1_Y3_7',
 'sh2_Y1_7',
 'sh2_Y3_7',
 'sh3_Y1_7',
 'sh3_Y3_7',
 'sh4_Y1_7',
 'sh4_Y3_7',
 'sh5_Y1_7',
 'sh5_Y3_7',
 'sh6_Y1_7',
 'sh6_Y3_7',
 'sh7_Y1_7',
 'sh7_Y3_7',
 'sh8_Y1_7',
 'sh8_Y3_7',
 'sh9_

In [32]:
# 1시간 이상 걸린다. 주의해서 돌리도록.
model_set = {}
for y in y_li:
    print('\n{}'.format(y))
    model = modelfit(alg, train, test, X_li, y)
    model_set[y] = model


sh1_Y1_1
Train Accuracy : 0.9087
Train AUC Score : 0.9700
Test  Accuracy : 0.5357
Test  AUC Score : 0.5423

sh1_Y3_1
Train Accuracy : 0.9970
Train AUC Score : 1.0000
Test  Accuracy : 0.4921
Test  AUC Score : 0.5158

sh2_Y1_1
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.4841
Test  AUC Score : 0.4777

sh2_Y3_1
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.4841
Test  AUC Score : 0.4777

sh3_Y1_1
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.4762
Test  AUC Score : 0.4162

sh3_Y3_1
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.4762
Test  AUC Score : 0.4162

sh4_Y1_1
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.5317
Test  AUC Score : 0.4564

sh4_Y3_1
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.5317
Test  AUC Score : 0.4564

sh5_Y1_1
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.5000
Test  AUC Score : 0.4654

sh5_Y3_1
Train Accuracy : 1

Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6905
Test  AUC Score : 0.5602

sh9_Y3_7
Train Accuracy : 0.9980
Train AUC Score : 1.0000
Test  Accuracy : 0.6349
Test  AUC Score : 0.5843

sh10_Y1_7
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6310
Test  AUC Score : 0.4964

sh10_Y3_7
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6825
Test  AUC Score : 0.6254

sh1_Y1_10
Train Accuracy : 0.9901
Train AUC Score : 0.9998
Test  Accuracy : 0.8254
Test  AUC Score : 0.9118

sh1_Y3_10
Train Accuracy : 0.9415
Train AUC Score : 0.9829
Test  Accuracy : 0.9167
Test  AUC Score : 0.9670

sh2_Y1_10
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8571
Test  AUC Score : 0.8881

sh2_Y3_10
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8730
Test  AUC Score : 0.9389

sh3_Y1_10
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.7381
Test  AUC Score : 0.7963

sh3_Y3_10
Train Accuracy : 1.0

Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6905
Test  AUC Score : 0.7437

sh7_Y3_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8651
Test  AUC Score : 0.9295

sh8_Y1_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6587
Test  AUC Score : 0.6175

sh8_Y3_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8611
Test  AUC Score : 0.9186

sh9_Y1_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6548
Test  AUC Score : 0.6264

sh9_Y3_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8611
Test  AUC Score : 0.9296

sh10_Y1_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6270
Test  AUC Score : 0.6600

sh10_Y3_25
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8333
Test  AUC Score : 0.9166

sh1_Y1_30
Train Accuracy : 0.9692
Train AUC Score : 0.9948
Test  Accuracy : 0.8770
Test  AUC Score : 0.9426

sh1_Y3_30
Train Accuracy : 

Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.7500
Test  AUC Score : 0.8218

sh5_Y3_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9167
Test  AUC Score : 0.9575

sh6_Y1_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.7183
Test  AUC Score : 0.7864

sh6_Y3_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9087
Test  AUC Score : 0.9615

sh7_Y1_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.7063
Test  AUC Score : 0.7041

sh7_Y3_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9206
Test  AUC Score : 0.9657

sh8_Y1_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.6984
Test  AUC Score : 0.6927

sh8_Y3_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9286
Test  AUC Score : 0.9583

sh9_Y1_45
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.7024
Test  AUC Score : 0.7357

sh9_Y3_45
Train Accuracy : 1.

Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8611
Test  AUC Score : 0.9120

sh3_Y3_65
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.9643
Test  AUC Score : 0.9895

sh4_Y1_65
Train Accuracy : 0.9821
Train AUC Score : 0.9987
Test  Accuracy : 0.8571
Test  AUC Score : 0.8926

sh4_Y3_65
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9683
Test  AUC Score : 0.9814

sh5_Y1_65
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8135
Test  AUC Score : 0.8577

sh5_Y3_65
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9563
Test  AUC Score : 0.9821

sh6_Y1_65
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8333
Test  AUC Score : 0.8491

sh6_Y3_65
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9643
Test  AUC Score : 0.9852

sh7_Y1_65
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.8413
Test  AUC Score : 0.8244

sh7_Y3_65
Train Accuracy : 1.

Train Accuracy : 0.9752
Train AUC Score : 0.9959
Test  Accuracy : 0.9683
Test  AUC Score : 0.9863

sh1_Y3_85
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.9603
Test  AUC Score : 0.9904

sh2_Y1_85
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9167
Test  AUC Score : 0.9480

sh2_Y3_85
Train Accuracy : 0.9762
Train AUC Score : 0.9977
Test  Accuracy : 0.9524
Test  AUC Score : 0.9633

sh3_Y1_85
Train Accuracy : 0.9980
Train AUC Score : 0.9999
Test  Accuracy : 0.8889
Test  AUC Score : 0.9427

sh3_Y3_85
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9325
Test  AUC Score : 0.9834

sh4_Y1_85
Train Accuracy : 0.9950
Train AUC Score : 0.9999
Test  Accuracy : 0.8810
Test  AUC Score : 0.9310

sh4_Y3_85
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9048
Test  AUC Score : 0.9821

sh5_Y1_85
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.8333
Test  AUC Score : 0.8894

sh5_Y3_85
Train Accuracy : 0.

Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.9325
Test  AUC Score : 0.9739

sh9_Y1_100
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.8929
Test  AUC Score : 0.9156

sh9_Y3_100
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9206
Test  AUC Score : 0.9672

sh10_Y1_100
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.8651
Test  AUC Score : 0.8115

sh10_Y3_100
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9524
Test  AUC Score : 0.9791

sh1_Y1_105
Train Accuracy : 0.9821
Train AUC Score : 0.9980
Test  Accuracy : 0.9643
Test  AUC Score : 0.9842

sh1_Y3_105
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.9841
Test  AUC Score : 0.9984

sh2_Y1_105
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9325
Test  AUC Score : 0.9570

sh2_Y3_105
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9881
Test  AUC Score : 0.9964

sh3_Y1_105
Train Ac

Train Accuracy : 0.9950
Train AUC Score : 0.9998
Test  Accuracy : 0.8770
Test  AUC Score : 0.9258

sh6_Y3_120
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9921
Test  AUC Score : 1.0000

sh7_Y1_120
Train Accuracy : 0.9980
Train AUC Score : 1.0000
Test  Accuracy : 0.8810
Test  AUC Score : 0.8962

sh7_Y3_120
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.9921
Test  AUC Score : 0.9997

sh8_Y1_120
Train Accuracy : 0.9921
Train AUC Score : 0.9974
Test  Accuracy : 0.9048
Test  AUC Score : 0.8879

sh8_Y3_120
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9921
Test  AUC Score : 1.0000

sh9_Y1_120
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.8651
Test  AUC Score : 0.7531

sh9_Y3_120
Train Accuracy : 1.0000
Train AUC Score : 1.0000
Test  Accuracy : 0.9881
Test  AUC Score : 1.0000

sh10_Y1_120
Train Accuracy : 0.9990
Train AUC Score : 1.0000
Test  Accuracy : 0.8690
Test  AUC Score : 0.7458

sh10_Y3_120
Train Ac

In [33]:
y1 = pd.DataFrame()
y3 = pd.DataFrame()
for idx, y in enumerate(y_li):
    acc = metrics.accuracy_score(test[y], model_set[y].predict(test[X_li]))
    auc = metrics.roc_auc_score(test[y], model_set[y].predict(test[X_li]))
    res = pd.DataFrame([acc,auc], index=['accuracy', 'AUC'], columns=[y])
    if idx%2==0:
        y1 = pd.concat((y1, res), axis=1)
    else:
        y3 = pd.concat((y3, res), axis=1)

In [36]:
y1.to_excel('./20181031 result/y1.xlsx')
y3.to_excel('./20181031 result/y3.xlsx')

In [37]:
with open('sp500_model(Not Feature Selected).pickle', 'wb') as handle:
    pickle.dump(model_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
pd.concat((pd.Series(X_li), pd.Series(model_set['sh1_Y3_5'].feature_importances_)),axis=1).sort_values(1, ascending=False)

,0,1
21,his_vol_21,0.011615
29,returnsOO1,0.009956
568,Y3_115_s10,0.009403
5,pct_change,0.009403
51,Y1_1_s2,0.008850
248,Y3_35_s10,0.008850
24,his_vol_90,0.008850
23,his_vol_60,0.008850
100,Y3_5_s6,0.008296
20,his_vol_7,0.008296


In [48]:
model_set['sh1_Y3_5'].feature_importances_

array([0.015625 , 0.       , 0.       , 0.       , 0.       , 0.0078125,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.0078125, 0.015625 , 0.       , 0.       ,
       0.       , 0.       , 0.0078125, 0.       , 0.0078125, 0.       ,
       0.       , 0.0078125, 0.       , 0.0078125, 0.       , 0.       ,
       0.       , 0.       , 0.03125  , 0.046875 , 0.0078125, 0.       ,
       0.       , 0.       , 0.0078125, 0.0078125, 0.       , 0.0078125,
       0.       , 0.       , 0.015625 , 0.       , 0.015625 , 0.015625 ,
       0.       , 0.       , 0.0078125, 0.0078125, 0.       , 0.0234375,
       0.0078125, 0.0390625, 0.03125  , 0.015625 , 0.0078125, 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.0078125, 0.       , 0.0234375,
       0.0390625, 0.0078125, 0.0078125, 0.015625 , 0.0078125, 0.       ,
       0.0078125, 0.       , 0.       , 0.       , 

In [44]:
pd.concat((X_li, model_set['sh1_Y3_5'].feature_importances_), axis=1)

TypeError: cannot concatenate object of type "<class 'list'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid